In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
# you need to uploud the IMDB dataset from kaggle to your drive
dataset = pd.read_csv("/content/IMDB Dataset.csv", on_bad_lines='skip', engine='python')
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [33]:
dataset.replace ({"sentiment":{"positive":1,"negative":0}}, inplace=True)


<ipython-input-33-a87709156a91>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace ({"sentiment":{"positive":1,"negative":0}}, inplace=True)


In [34]:
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=30)
print(train_data.shape, test_data.shape)

(40000, 2) (10000, 2)


In [35]:
tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(train_data["review"])

x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)



y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [36]:
embedding_dim=32
vocab_size=6000

model = Sequential([
    Embedding(vocab_size, embedding_dim, name="embedding"),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [37]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [38]:
history = model.fit( x_train,
                    y_train,
                    epochs=4,
                    batch_size=50,
                    validation_split=0.2)

Epoch 1/4
640/640 ━━━━━━━━━━━━━━━━━━━━ 141s 214ms/step - accuracy: 0.6970 - loss: 0.5335 - val_accuracy: 0.8618 - val_loss: 0.3172
Epoch 2/4
640/640 ━━━━━━━━━━━━━━━━━━━━ 142s 214ms/step - accuracy: 0.8942 - loss: 0.2660 - val_accuracy: 0.8852 - val_loss: 0.2889
Epoch 3/4
640/640 ━━━━━━━━━━━━━━━━━━━━ 136s 213ms/step - accuracy: 0.9066 - loss: 0.2361 - val_accuracy: 0.8839 - val_loss: 0.2770
Epoch 4/4
640/640 ━━━━━━━━━━━━━━━━━━━━ 140s 210ms/step - accuracy: 0.9296 - loss: 0.1889 - val_accuracy: 0.8842 - val_loss: 0.2784


In [39]:
def predict_sentiment(review):

  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)

  prediction = model.predict(padded_sequence)
  print(prediction)

  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [40]:
while True:
  new_review = input("Enter a movie review (or type 'exit' to quit): ")
  if new_review.lower() == "exit":
    break
  sentiment = predict_sentiment(new_review)
  print("Sentiment of the review is:", sentiment)

Enter a movie review (or type 'exit' to quit): i love the movie
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step
[[0.8130258]]
Sentiment of the review is: positive
Enter a movie review (or type 'exit' to quit): this was a very bad movie
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
[[0.21604414]]
Sentiment of the review is: negative
Enter a movie review (or type 'exit' to quit): i hate this movie
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
[[0.48545203]]
Sentiment of the review is: negative
Enter a movie review (or type 'exit' to quit): exit
